In [124]:
driver.close()

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import datetime as dt
import pandas as pd
from collections import Counter
import os
# os.chdir('/content/drive/MyDrive/015GithubRepos/Class_DatabaseSystem')

In [ ]:
# data_PATH = '/content/drive/MyDrive/015GithubRepos/Class_DatabaseSystem'

In [19]:
driver = webdriver.Chrome('J:/내 드라이브/015GithubRepos/Class_DatabaseSystem/chromedriver.exe')
data_PATH = 'J:/내 드라이브/015GithubRepos/Class_DatabaseSystem/'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
urls = [
    'https://search.musinsa.com/category/001001', # 반팔 티셔츠
    'https://search.musinsa.com/category/003002', # 데님팬츠
    'https://search.musinsa.com/category/003008', # 슈트 팬츠/슬랙스
    'https://search.musinsa.com/category/001002', # 셔츠/블라우스
    'https://search.musinsa.com/category/003004', # 트레이닝/조거 팬츠
    'https://search.musinsa.com/category/003009' # 숏 팬츠
#     'https://search.musinsa.com/category/005004', # 샌들
]

In [21]:
# 무신사 홈페이지로 이동한 다음 우측 하단 '전체', '남성', '여성' 중 '여성'을
# 클릭해야함
# 홈페이지로 들어가고 끝나면, 무신사와 우신사 url이 같아서 무신사로 들어가게됨
driver.get('https://store.musinsa.com/app/')
driver.find_element_by_css_selector('button.global-filter__button.global-filter__button--wusinsa').click()

In [102]:
# 수집할 데이터 리스트들 ##############################
product_names = []
categories = []

retypes = []

# purchase size, sex, height, weight
meta_pshw = []
meta_sizes = []
meta_brights = []
meta_colors = []
meta_thicks = []

rvs = []
scores = []
helpfuls = []
uploadAts = []

collectAts = []
############################## 수집할 데이터 리스트들 #

# pattern for max page
p_maxp = re.compile(r'[\d,]+')

n = 10
nn = 3200


---

# 뜯어서 검증

In [24]:
cateurl

'https://search.musinsa.com/category/001001'

In [25]:
# ii : 카테고리별 수집 개수 cnt
# i : product별 수집 개수 cnt
ii = 0

# 카테고리로 이동(ex. 반팔티셔)
driver.get(cateurl)
time.sleep(3)

dom = BeautifulSoup(driver.page_source, 'html.parser')
time.sleep(0.5)

#  카테고리(반팔티셔츠)별 제품들의 url들 구하기

driver.get(cateurl)
dom = BeautifulSoup(driver.page_source, 'html.parser')
time.sleep(0.5)
cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
product_urls = [_['href'] for _ in cate_products]

for _ in range(4, 13):
    driver.find_element_by_css_selector(f'div.sorter-box.box div.pagination div.wrapper a:nth-of-type({_})').click()
    time.sleep(0.5)
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    page_cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
    page_product_urls = [_['href'] for _ in page_cate_products]
    product_urls += page_product_urls
#     print(f'_ : {_}, page : {_-2}')


# product 개수 세기
pn = 0

In [26]:
for product_url in product_urls:
    break

In [27]:
product_url

'https://store.musinsa.com/app/goods/324416'

In [78]:
product_url = 'https://store.musinsa.com/app/goods/558627'

In [79]:
pn += 1
# product로 이동
driver.get(product_url)
time.sleep(3)

dom = BeautifulSoup(driver.page_source, 'html.parser')
time.sleep(0.5)

product_name = dom.select_one('span.product_title em').text.strip()
category = dom.select_one('p.item_categories a:nth-of-type(2)').text.strip()


numofrv = dom.select_one('h3.title-box.font-mss span#review_total').text.strip()
prdid = product_url.split('/')[-1]
print(f'상품아이디: {prdid}, 본 상품 총 후기 개수 : {numofrv}')
# i == n이면 빠져나오기 위함
# product별 최대 100개씩
i = 0


# review types 1 : 스타일 후기, 2 : 상품사진후기, 3 : 일반 후기
# 체험단후기가 있는 경우, 2 : 스타일 후기, 3 : 상품사진후기, 4 : 일반 후기
if len(dom.select('div#estimateBox div.tabBox > ul.snb > li')) == 4:
    rvtypes = {2 : '스타일 후기', 3: '상품 사진 후기', 4: '일반 후기'}
else:
    rvtypes = {1 : '스타일 후기', 2: '상품 사진 후기', 3: '일반 후기'}

상품아이디: 558627, 본 상품 총 후기 개수 : 43


In [68]:
len(dom)

1

In [99]:
rvtype = 3

In [100]:

# review type 화면으로 이동
# driver.find_element_by_css_selector(f'div#estimateBox div.tabBox > ul.snb > li:nth-of-type({rvtype})').click()
# time.sleep(1)

# # dom 불러오기
# dom = BeautifulSoup(driver.page_source, 'html.parser')
# time.sleep(0.5)

# page description : 최대 몇 페이지까지 있는지 텍스트 ('000페이지 중 000페이지')
# maxp : 해당 상품의 마지막 페이지 숫자
if dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg'):
    pagedes = dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg').text.strip()
    maxp = int(re.search(p_maxp, pagedes).group(0).replace(',', ''))

    # curr_num : 현재 페이지
    curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
    curr_num = int(curr_num)
else:  # 후기가 없는 경우
    maxp = 0


print(f'{rvtypes[rvtype]} max page : {maxp}')



# 일반 후기까지 다 했거나, i==100이면 리뷰타입 볼 필요 없이 다음 product로 넘어간다
if i == n or ii >= nn:
    print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
    

####################################### review type별 수집 #

일반 후기 max page : 2


In [101]:
maxp

2

In [ ]:
rvtype = ['style_estimate_list', 'photo_estimate_list', 'estimate_list']

In [72]:
for rvbox in dom.select(f'div#estimateBox div.tabBox div#{rvtype[2]} div.review-list'):
    print(rvbox.select_one('div.review-contents__text').text.strip())
    print()

S인데도 허리랑 허벅지쪽이 커요. 그래도 길이는 짧지 않아서 좋고, 만약 핏이 딱 떨어지는 숏팬츠를 원한다면 추천하지 않아요.

무난하게 입기 좋습니다. 다만 원단이 좀 얇아서 말려올라가긴 했네요.

길이가 막 길지도 짧지도 않고 적당해서 좋아요 또 예뻐요

색감 진짜이뻐요ㅠㅠ̑̈ 길이도 적당하고 핏도 딱 떨어지는게 굿굿 이번여름 자주입었어용ㅎㅅㅎ

평소에 입는 사이즈보다 약간 큰 편이며 적당한 하이웨이스트 스타일임

완전 아주 밝은 연청색입니다. 얇아서 여름에 시원하게 입을 수 있을것 같아요

편하고 핏도 이쁘구 시원한 소재구
완벽해요 ㅠㅠ♡ 올해 바지 쇼핑 성공!
이번에 첨 안 브랜드인데 더 사고싶네요

계속 입다보면 너무 a라인 같긴한데 그래두 예뻐용

저렴한 가격 대비
원단 도톰하고 색감 너무 시원해보여요

주름지는거는 재질이 얇아서 그런거 같아요 개인적으로 허리도 컸어요 그거 빼고는 괜찮았고요



In [39]:
rvbox

<div class="review-list">
<div class="review-profile">
<a class="review-profile__link" href="/app/reviews/lists?hash_id=a37f67d304d7946dfecf3c47c9a4f281&amp;type=style">
<img alt="" class="review-profile__image" src="//image.msscdn.net/mfile_s01/_simbols/_basic/t.png"/>
</a>
<div class="review-profile__text-wrap">
<div class="review-profile__text">
<p class="review-profile__name">
                                                                                                        LV 5
                                                        꽈당쑤
                                                            </p>
<p class="review-profile__date">
                                            2020.08.06
                                    </p>
</div>
<div class="review-profile__information">
<a class="report" onclick="reportReviewsPop('/app/cs/report_pop/EST/11057287')" title="신고">신고</a>
</div>
</div>
</div>
<div class="review-goods-information">
<div class="review-goods-information__thumbnai

In [40]:
rvbox.select_one('span.review-list__rating__active').get('style')[-3] in ['2', '4', '6', '8']

False

In [36]:
# paging ##################################
while maxp:

    # dom 새로 불러오기
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)

    # 현재 페이지 구하기
    curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
    curr_num = int(curr_num)
    print(f'current page number : {curr_num}, length of rvs : {len(rvs)}')

    # list of pages : 페이지 리스트
    lp = [int(_.text.strip()) if _.text.strip() != '' else 0 for _ in dom.select('div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a')]
    
    # 한 페이지에서 데이터 수집 ################################
    for rvbox in dom.select('div#estimateBox div.tabBox div#style_estimate_list div.review-list'):

        if rvbox.select_one('span.review-list__rating__active').get('style')[-3] in ['2', '4', '6', '8']:

            product_names.append(product_name)
            categories.append(category)

            retypes.append(rvtypes[rvtype])

            # purchase size, sex, height, weight
            if rvbox.select_one('p.review-profile__body_information'):
                meta_pshw.append(rvbox.select_one('p.review-profile__body_information').text.strip())
            else:
                meta_pshw.append('')


            # prd-level-each
            if rvbox.select_one('div.review-evaluation'):
                meta_sizes.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(1)').text.strip())
                meta_brights.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(2)').text.strip())
                meta_colors.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(3)').text.strip())
                meta_thicks.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(4)').text.strip())
            else:
                meta_sizes.append('')
                meta_brights.append('')
                meta_colors.append('')
                meta_thicks.append('')


            rvs.append(rvbox.select_one('div.review-contents__text').text.strip())
            scores.append(rvbox.select_one('span.review-list__rating__active').get('style'))
            helpfuls.append(int(rvbox.select_one('span.review-evaluation-button__count').text))
            uploadAts.append(rvbox.select_one('p.review-profile__date').text.strip())

            collectAts.append(dt.datetime.today())

            i += 1
            ii += 1

        if i == n or ii >= nn:
            break
    ################################ 한 페이지에서 데이터 수집 #

    if i == n or ii >= nn:
        break

    # 현재 페이지가 최종 페이지인지 확인
    if maxp == curr_num:
        print(f'{rvtypes[rvtype]} maxpage도달')
        print()
        if rvtypes[rvtype] == '일반 후기':
            print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
        break
    else: # 최종 페이지가 아니면 다음장 (또는 다음 5페이지 화면)으로 이동
        time.sleep(0.1)
        driver.find_element_by_css_selector(f'div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a:nth-of-type({lp.index(curr_num) + 2})').click()

    time.sleep(0.5)
################################## paging #

current page number : 1, length of rvs : 0


In [23]:
for cateurl in urls:

    
    # ii : 카테고리별 수집 개수 cnt
    # i : product별 수집 개수 cnt
    ii = 0
    
    # 카테고리로 이동(ex. 반팔티셔)
    driver.get(cateurl)
    time.sleep(3)
    
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)
    
    #  카테고리(반팔티셔츠)별 제품들의 url들 구하기
    
    driver.get(cateurl)
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)
    cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
    product_urls = [_['href'] for _ in cate_products]

    for _ in range(4, 13):
        driver.find_element_by_css_selector(f'div.sorter-box.box div.pagination div.wrapper a:nth-of-type({_})').click()
        time.sleep(0.5)
        dom = BeautifulSoup(driver.page_source, 'html.parser')
        page_cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
        page_product_urls = [_['href'] for _ in page_cate_products]
        product_urls += page_product_urls
    #     print(f'_ : {_}, page : {_-2}')

    
    # product 개수 세기
    pn = 0
    
    # product별 url로 이동 #########################################
    for product_url in product_urls:
        pn += 1
        # product로 이동
        driver.get(product_url)
        time.sleep(3)
        
        dom = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(0.5)
    
        product_name = dom.select_one('span.product_title em').text.strip()
        category = dom.select_one('p.item_categories a:nth-of-type(2)').text.strip()
        
        
        numofrv = dom.select_one('h3.title-box.font-mss span#review_total').text.strip()
        prdid = product_url.split('/')[-1]
        print(f'상품아이디: {prdid}, 본 상품 총 후기 개수 : {numofrv}')
        # i == n이면 빠져나오기 위함
        # product별 최대 100개씩
        i = 0
        

        # review types 1 : 스타일 후기, 2 : 상품사진후기, 3 : 일반 후기
        # 체험단후기가 있는 경우, 2 : 스타일 후기, 3 : 상품사진후기, 4 : 일반 후기
        if len(dom.select('div#estimateBox div.tabBox > ul.snb > li')) == 4:
            rvtypes = {2 : 'style_estimate_list', 3: 'photo_estimate_list', 4: 'estimate_list'}
        else:
            rvtypes = {1 : 'style_estimate_list', 2: 'photo_estimate_list', 3: 'estimate_list'}
        
        
        
        # review type별 수집 #######################################
        for rvtype in rvtypes:
            
            # review type 화면으로 이동
#             driver.find_element_by_css_selector(f'div#estimateBox div.tabBox > ul.snb > li:nth-of-type({rvtype})').click()
#             time.sleep(1)
    
            # dom 불러오기
#             dom = BeautifulSoup(driver.page_source, 'html.parser')
#             time.sleep(0.5)

            # page description : 최대 몇 페이지까지 있는지 텍스트 ('000페이지 중 000페이지')
            # maxp : 해당 상품의 마지막 페이지 숫자
            if dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg'):
                pagedes = dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg').text.strip()
                maxp = int(re.search(p_maxp, pagedes).group(0).replace(',', ''))
                
                # curr_num : 현재 페이지
                curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
                curr_num = int(curr_num)
            else:  # 후기가 없는 경우
                maxp = 0


            print(f'{rvtypes[rvtype]} max page : {maxp}')
    
            # paging ##################################
            while maxp:

                # dom 새로 불러오기
                dom = BeautifulSoup(driver.page_source, 'html.parser')
                time.sleep(0.5)
                
                # 현재 페이지 구하기
                curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
                curr_num = int(curr_num)
                print(f'current page number : {curr_num}, length of rvs : {len(rvs)}')
    
                # list of pages : 페이지 리스트
                lp = [int(_.text.strip()) if _.text.strip() != '' else 0 for _ in dom.select('div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a')]
    
                # 한 페이지에서 데이터 수집 ################################
                for rvbox in dom.select(f'div#estimateBox div.tabBox div#{rvtypes[rvtype]} div.review-list'):
                    
                    if rvbox.select_one('span.review-list__rating__active').get('style')[-3] in ['2', '4', '6', '8']:
                    
                        product_names.append(product_name)
                        categories.append(category)

                        retypes.append(rvtypes[rvtype])

                        # purchase size, sex, height, weight
                        if rvbox.select_one('p.review-profile__body_information'):
                            meta_pshw.append(rvbox.select_one('p.review-profile__body_information').text.strip())
                        else:
                            meta_pshw.append('')


                        # prd-level-each
                        if rvbox.select_one('div.review-evaluation'):
                            meta_sizes.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(1)').text.strip())
                            meta_brights.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(2)').text.strip())
                            meta_colors.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(3)').text.strip())
                            meta_thicks.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(4)').text.strip())
                        else:
                            meta_sizes.append('')
                            meta_brights.append('')
                            meta_colors.append('')
                            meta_thicks.append('')


                        rvs.append(rvbox.select_one('div.review-contents__text').text.strip())
                        scores.append(rvbox.select_one('span.review-list__rating__active').get('style'))
                        helpfuls.append(int(rvbox.select_one('span.review-evaluation-button__count').text))
                        uploadAts.append(rvbox.select_one('p.review-profile__date').text.strip())

                        collectAts.append(dt.datetime.today())

                        i += 1
                        ii += 1

                    if i == n or ii >= nn:
                        break
                ################################ 한 페이지에서 데이터 수집 #
                
                if i == n or ii >= nn:
                    break
                
                # 현재 페이지가 최종 페이지인지 확인
                if maxp == curr_num:
                    print(f'{rvtypes[rvtype]} maxpage도달')
                    print()
                    if rvtypes[rvtype] == '일반 후기':
                        print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
                    break
                else: # 최종 페이지가 아니면 다음장 (또는 다음 5페이지 화면)으로 이동
                    time.sleep(0.1)
                    driver.find_element_by_css_selector(f'div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a:nth-of-type({lp.index(curr_num) + 2})').click()
    
                time.sleep(0.5)
            ################################## paging #
            
            # 일반 후기까지 다 했거나, i==100이면 리뷰타입 볼 필요 없이 다음 product로 넘어간다
            if i == n or ii >= nn:
                print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
                break
        
        ####################################### review type별 수집 #
        
        print(f'product 1개 끝*******************************')
        print()
        
        # 한 카테고리에서 nn개 이상이 쌓이면 다음 카테고리로
        if ii >= nn:
            print(f'category 1개 완료, {category}, 데이터 길이 : {len(product_names)}')
            print()
            break
    ######################################### product별 url로 이동 #
    
    
############################################# category별 url로 이동 #

# 뜯어서 검증 끝

---

# 실제 돌리는 전체코드

In [116]:
# 수집할 데이터 리스트들 ##############################
product_names = []
categories = []

retypes = []

# purchase size, sex, height, weight
meta_pshw = []
meta_sizes = []
meta_brights = []
meta_colors = []
meta_thicks = []

rvs = []
scores = []
helpfuls = []
uploadAts = []

collectAts = []
############################## 수집할 데이터 리스트들 #

# pattern for max page
p_maxp = re.compile(r'[\d,]+')

n = 100
nn = 5000

# category별 url로 이동 #############################################
for cateurl in urls:
    
    
    # ii : 카테고리별 수집 개수 cnt
    # i : product별 수집 개수 cnt
    ii = 0
    
    # 카테고리로 이동(ex. 반팔티셔)
    driver.get(cateurl)
    time.sleep(3)
    
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)
    
    #  카테고리(반팔티셔츠)별 제품들의 url들 구하기
    
    driver.get(cateurl)
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(0.5)
    cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
    product_urls = [_['href'] for _ in cate_products]
    
    print(f'카테고리 url : {cateurl}')
    print(f'아이템 url 수집 중...')
    for _ in range(4, 13):
        driver.find_element_by_css_selector(f'div.sorter-box.box div.pagination div.wrapper a:nth-of-type({_})').click()
        time.sleep(0.5)
        dom = BeautifulSoup(driver.page_source, 'html.parser')
        page_cate_products = dom.select('ul#searchList li div.article_info p.list_info a')
        page_product_urls = [_['href'] for _ in page_cate_products]
        product_urls += page_product_urls
    #     print(f'_ : {_}, page : {_-2}')

    
    # product 개수 세기
    pn = 0
    
    # product별 url로 이동 #########################################
    for product_url in product_urls:
        pn += 1
        # product로 이동
        driver.get(product_url)
        time.sleep(3)
        
        dom = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(0.5)
    
        product_name = dom.select_one('span.product_title em').text.strip()
        category = dom.select_one('p.item_categories a:nth-of-type(2)').text.strip()
        
        
        numofrv = dom.select_one('h3.title-box.font-mss span#review_total').text.strip()
        prdid = product_url.split('/')[-1]
        print(f'상품아이디: {prdid}, 본 상품 총 후기 개수 : {numofrv}')
        # i == n이면 빠져나오기 위함
        # product별 최대 100개씩
        i = 0
        

        # review types 1 : 스타일 후기, 2 : 상품사진후기, 3 : 일반 후기
        # 체험단후기가 있는 경우, 2 : 스타일 후기, 3 : 상품사진후기, 4 : 일반 후기
        if len(dom.select('div#estimateBox div.tabBox > ul.snb > li')) == 4:
            rvtypes = {2 : 'style_estimate_list', 3: 'photo_estimate_list', 4: 'estimate_list'}
        else:
            rvtypes = {1 : 'style_estimate_list', 2: 'photo_estimate_list', 3: 'estimate_list'}
        
        # review type별 수집 #######################################
        for rvtype in rvtypes:
            
            # review type 화면으로 이동
            driver.find_element_by_css_selector(f'div#estimateBox div.tabBox > ul.snb > li:nth-of-type({rvtype})').click()
            time.sleep(1)
    
            # dom 불러오기
            dom = BeautifulSoup(driver.page_source, 'html.parser')
            time.sleep(0.5)

            # page description : 최대 몇 페이지까지 있는지 텍스트 ('000페이지 중 000페이지')
            # maxp : 해당 상품의 마지막 페이지 숫자
            if dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg'):
                pagedes = dom.select_one('div#estimateBox div.tabBox div.active div.box_page_msg').text.strip()
                maxp = int(re.search(p_maxp, pagedes).group(0).replace(',', ''))
                
                # curr_num : 현재 페이지
                curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
                curr_num = int(curr_num)
            else:  # 후기가 없는 경우
                maxp = 0


            print(f'{rvtypes[rvtype]} max page : {maxp}')
    
            # paging ##################################
            while maxp:

                # dom 새로 불러오기
                dom = BeautifulSoup(driver.page_source, 'html.parser')
                time.sleep(0.5)
                
                # 현재 페이지 구하기
                curr_num = dom.select_one('div#estimateBox div.tabBox div.active a.active').text.strip()
                curr_num = int(curr_num)
                print(f'current page number : {curr_num}, length of rvs : {len(rvs)}')
    
                # list of pages : 페이지 리스트
                lp = [int(_.text.strip()) if _.text.strip() != '' else 0 for _ in dom.select('div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a')]
    
                # 한 페이지에서 데이터 수집 ################################
                for rvbox in dom.select(f'div#estimateBox div.tabBox div#{rvtypes[rvtype]} div.review-list'):
                    
                    if rvbox.select_one('span.review-list__rating__active').get('style')[-3] not in ['2', '4', '6', '8']:
                    
                        product_names.append(product_name)
                        categories.append(category)

                        retypes.append(rvtypes[rvtype])

                        # purchase size, sex, height, weight
                        if rvbox.select_one('p.review-profile__body_information'):
                            meta_pshw.append(rvbox.select_one('p.review-profile__body_information').text.strip())
                        else:
                            meta_pshw.append('')


                        # prd-level-each
                        if rvbox.select_one('div.review-evaluation'):
                            meta_sizes.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(1)').text.strip())
                            meta_brights.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(2)').text.strip())
                            meta_colors.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(3)').text.strip())
                            meta_thicks.append(rvbox.select_one('div.review-evaluation ul li:nth-of-type(4)').text.strip())
                        else:
                            meta_sizes.append('')
                            meta_brights.append('')
                            meta_colors.append('')
                            meta_thicks.append('')


                        rvs.append(rvbox.select_one('div.review-contents__text').text.strip())
                        scores.append(rvbox.select_one('span.review-list__rating__active').get('style'))
                        helpfuls.append(int(rvbox.select_one('span.review-evaluation-button__count').text))
                        uploadAts.append(rvbox.select_one('p.review-profile__date').text.strip())

                        collectAts.append(dt.datetime.today())

                        i += 1
                        ii += 1

                    if i == n or ii >= nn:
                        break
                ################################ 한 페이지에서 데이터 수집 #
                
                if i == n or ii >= nn:
                    break
                
                # 현재 페이지가 최종 페이지인지 확인
                if maxp == curr_num:
                    print(f'{rvtypes[rvtype]} maxpage도달')
                    print()
                    if rvtypes[rvtype] == '일반 후기':
                        print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
                    break
                else: # 최종 페이지가 아니면 다음장 (또는 다음 5페이지 화면)으로 이동
                    time.sleep(0.1)
                    driver.find_element_by_css_selector(f'div#estimateBox div.tabBox div.active div.pagination.textRight div.wrapper a:nth-of-type({lp.index(curr_num) + 2})').click()
    
                time.sleep(0.5)
            ################################## paging #
            
            # 일반 후기까지 다 했거나, i==100이면 리뷰타입 볼 필요 없이 다음 product로 넘어간다
            if i == n or ii >= nn:
                print(f'product {pn}개째 완료, {product_name}, 데이터 길이 : {len(product_names)}')
                break
        
        ####################################### review type별 수집 #
        
        print(f'product 1개 끝*******************************')
        print()
        
        # 한 카테고리에서 nn개 이상이 쌓이면 다음 카테고리로
        if ii >= nn:
            print(f'category 1개 완료, {category}, 데이터 길이 : {len(product_names)}')
            print()
            break
    ######################################### product별 url로 이동 #
    
    
############################################# category별 url로 이동 #

상품아이디: 324416, 본 상품 총 후기 개수 : 29,026
style_estimate_list max page : 246
current page number : 1, length of rvs : 0
current page number : 2, length of rvs : 10
current page number : 3, length of rvs : 16
current page number : 4, length of rvs : 24
current page number : 5, length of rvs : 32
current page number : 6, length of rvs : 41
current page number : 7, length of rvs : 50
current page number : 8, length of rvs : 60
current page number : 9, length of rvs : 68
current page number : 10, length of rvs : 77
current page number : 11, length of rvs : 85
current page number : 12, length of rvs : 94
product 1개째 완료, [세트상품] 더블 코튼 롱 티셔츠, 데이터 길이 : 100
product 1개 끝*******************************

상품아이디: 1417691, 본 상품 총 후기 개수 : 15,615
style_estimate_list max page : 131
current page number : 1, length of rvs : 100
current page number : 2, length of rvs : 109
current page number : 3, length of rvs : 114
current page number : 4, length of rvs : 123
current page number : 5, length of rvs : 131
curren

current page number : 9, length of rvs : 1168
current page number : 10, length of rvs : 1177
current page number : 11, length of rvs : 1186
current page number : 12, length of rvs : 1194
product 12개째 완료, 어센틱 로고 티셔츠 블랙, 데이터 길이 : 1200
product 1개 끝*******************************

상품아이디: 1884943, 본 상품 총 후기 개수 : 8,943
style_estimate_list max page : 131
current page number : 1, length of rvs : 1200
current page number : 2, length of rvs : 1208
current page number : 3, length of rvs : 1218
current page number : 4, length of rvs : 1226
current page number : 5, length of rvs : 1234
current page number : 6, length of rvs : 1244
current page number : 7, length of rvs : 1252
current page number : 8, length of rvs : 1262
current page number : 9, length of rvs : 1272
current page number : 10, length of rvs : 1282
current page number : 11, length of rvs : 1289
current page number : 12, length of rvs : 1298
product 13개째 완료, 미시간 울버린 피그먼트  반팔티 블랙차콜, 데이터 길이 : 1300
product 1개 끝****************************

상품아이디: 1846930, 본 상품 총 후기 개수 : 7,556
style_estimate_list max page : 113
current page number : 1, length of rvs : 2300
current page number : 2, length of rvs : 2309
current page number : 3, length of rvs : 2317
current page number : 4, length of rvs : 2327
current page number : 5, length of rvs : 2335
current page number : 6, length of rvs : 2345
current page number : 7, length of rvs : 2352
current page number : 8, length of rvs : 2359
current page number : 9, length of rvs : 2367
current page number : 10, length of rvs : 2376
current page number : 11, length of rvs : 2385
current page number : 12, length of rvs : 2391
product 24개째 완료, EFF 베츠 반팔 티셔츠 블루, 데이터 길이 : 2400
product 1개 끝*******************************

상품아이디: 1417715, 본 상품 총 후기 개수 : 6,371
style_estimate_list max page : 76
current page number : 1, length of rvs : 2400
current page number : 2, length of rvs : 2410
current page number : 3, length of rvs : 2420
current page number : 4, length of rvs : 2430
current page number : 5,

current page number : 9, length of rvs : 3414
current page number : 10, length of rvs : 3420
current page number : 11, length of rvs : 3430
current page number : 12, length of rvs : 3440
product 35개째 완료, T-Logo Tee White (003), 데이터 길이 : 3443
product 1개 끝*******************************

상품아이디: 1042882, 본 상품 총 후기 개수 : 16,007
style_estimate_list max page : 107
current page number : 1, length of rvs : 3443
current page number : 2, length of rvs : 3452
current page number : 3, length of rvs : 3458
current page number : 4, length of rvs : 3467
current page number : 5, length of rvs : 3477
current page number : 6, length of rvs : 3484
current page number : 7, length of rvs : 3493
current page number : 8, length of rvs : 3502
current page number : 9, length of rvs : 3512
current page number : 10, length of rvs : 3519
current page number : 11, length of rvs : 3525
current page number : 12, length of rvs : 3534
product 36개째 완료, 유니섹스 오버핏 어스넷 로고 티-화이트, 데이터 길이 : 3543
product 1개 끝*******************

current page number : 1, length of rvs : 4543
current page number : 2, length of rvs : 4553
current page number : 3, length of rvs : 4562
current page number : 4, length of rvs : 4569
current page number : 5, length of rvs : 4578
current page number : 6, length of rvs : 4586
current page number : 7, length of rvs : 4596
current page number : 8, length of rvs : 4606
current page number : 9, length of rvs : 4616
current page number : 10, length of rvs : 4626
current page number : 11, length of rvs : 4635
current page number : 12, length of rvs : 4641
product 47개째 완료, MARITHE W EMBROIDERY TEE white, 데이터 길이 : 4643
product 1개 끝*******************************

상품아이디: 1975160, 본 상품 총 후기 개수 : 2,329
style_estimate_list max page : 22
current page number : 1, length of rvs : 4643
current page number : 2, length of rvs : 4651
current page number : 3, length of rvs : 4660
current page number : 4, length of rvs : 4669
current page number : 5, length of rvs : 4678
current page number : 6, length of r

...

current page number : 4, length of rvs : 26726
current page number : 5, length of rvs : 26734
current page number : 6, length of rvs : 26743
current page number : 7, length of rvs : 26750
current page number : 8, length of rvs : 26754
current page number : 9, length of rvs : 26762
current page number : 10, length of rvs : 26770
current page number : 11, length of rvs : 26779
current page number : 12, length of rvs : 26785
current page number : 13, length of rvs : 26791
current page number : 14, length of rvs : 26798
product 51개째 완료, BASIC HALF PANTS (KHAKI) [LRPMCPH701MKHA], 데이터 길이 : 26800
product 1개 끝*******************************

상품아이디: 2181415, 본 상품 총 후기 개수 : 0
style_estimate_list max page : 0
photo_estimate_list max page : 0
estimate_list max page : 0
product 1개 끝*******************************

상품아이디: 2181539, 본 상품 총 후기 개수 : 0
style_estimate_list max page : 0
photo_estimate_list max page : 0
estimate_list max page : 0
product 1개 끝*******************************

상품아이디: 1852956, 

current page number : 5, length of rvs : 27540
current page number : 6, length of rvs : 27549
current page number : 7, length of rvs : 27555
current page number : 8, length of rvs : 27562
product 62개째 완료, RLJP607 보드쇼츠 - 2 PACK, 데이터 길이 : 27570
product 1개 끝*******************************

상품아이디: 1432207, 본 상품 총 후기 개수 : 1,548
style_estimate_list max page : 15
current page number : 1, length of rvs : 27570
current page number : 2, length of rvs : 27579
current page number : 3, length of rvs : 27585
current page number : 4, length of rvs : 27591
current page number : 5, length of rvs : 27598
current page number : 6, length of rvs : 27607
current page number : 7, length of rvs : 27613
current page number : 8, length of rvs : 27623
current page number : 9, length of rvs : 27630
current page number : 10, length of rvs : 27638
current page number : 11, length of rvs : 27646
current page number : 12, length of rvs : 27656
current page number : 13, length of rvs : 27664
product 63개째 완료, 우먼즈 라운지 쇼

style_estimate_list max page : 0
photo_estimate_list max page : 0
estimate_list max page : 0
product 1개 끝*******************************

상품아이디: 2178989, 본 상품 총 후기 개수 : 0
style_estimate_list max page : 0
photo_estimate_list max page : 0
estimate_list max page : 0
product 1개 끝*******************************

상품아이디: 1517257, 본 상품 총 후기 개수 : 9
style_estimate_list max page : 1
current page number : 1, length of rvs : 28400
style_estimate_list maxpage도달

photo_estimate_list max page : 1
current page number : 1, length of rvs : 28401
photo_estimate_list maxpage도달

estimate_list max page : 1
current page number : 1, length of rvs : 28403
estimate_list maxpage도달

product 1개 끝*******************************

상품아이디: 2014535, 본 상품 총 후기 개수 : 14
style_estimate_list max page : 1
current page number : 1, length of rvs : 28408
style_estimate_list maxpage도달

photo_estimate_list max page : 1
current page number : 1, length of rvs : 28409
photo_estimate_list maxpage도달

estimate_list max page : 1
current p

current page number : 4, length of rvs : 28941
current page number : 5, length of rvs : 28948
estimate_list maxpage도달

product 1개 끝*******************************

상품아이디: 1461441, 본 상품 총 후기 개수 : 290
style_estimate_list max page : 3
current page number : 1, length of rvs : 28952
current page number : 2, length of rvs : 28960
current page number : 3, length of rvs : 28966
style_estimate_list maxpage도달

photo_estimate_list max page : 8
current page number : 1, length of rvs : 28971
current page number : 2, length of rvs : 28980
current page number : 3, length of rvs : 28986
current page number : 4, length of rvs : 28993
current page number : 5, length of rvs : 28998
current page number : 6, length of rvs : 29004
current page number : 7, length of rvs : 29013
current page number : 8, length of rvs : 29019
photo_estimate_list maxpage도달

estimate_list max page : 19
current page number : 1, length of rvs : 29024
current page number : 2, length of rvs : 29032
current page number : 3, length of

current page number : 3, length of rvs : 29605
current page number : 4, length of rvs : 29615
current page number : 5, length of rvs : 29624
current page number : 6, length of rvs : 29633
current page number : 7, length of rvs : 29642
current page number : 8, length of rvs : 29649
product 104개째 완료, 기어 쉘 쇼츠 Black, 데이터 길이 : 29651
product 1개 끝*******************************

상품아이디: 1376514, 본 상품 총 후기 개수 : 389
style_estimate_list max page : 4
current page number : 1, length of rvs : 29651
current page number : 2, length of rvs : 29659
current page number : 3, length of rvs : 29667
current page number : 4, length of rvs : 29675
style_estimate_list maxpage도달

photo_estimate_list max page : 10
current page number : 1, length of rvs : 29682
current page number : 2, length of rvs : 29687
current page number : 3, length of rvs : 29694
current page number : 4, length of rvs : 29701
current page number : 5, length of rvs : 29708
current page number : 6, length of rvs : 29715
current page number : 

In [117]:
df = pd.DataFrame(
    {
     'product_names' :product_names,
     
     'categories' :categories,
     'retypes' :retypes,
     'rvs' :rvs,

     'meta_pshw': meta_pshw,
     'meta_sizes': meta_sizes,
     'meta_brights': meta_brights,
     'meta_colors': meta_colors,
     'meta_thicks': meta_thicks,
     
     'scores' :scores,
     'helpfuls' :helpfuls,
     'uploadAts' :uploadAts,
     'collectAts' :collectAts
     })

In [123]:
df.to_csv('crawling_20211110_positive.csv')

In [118]:
df

,product_names,categories,retypes,rvs,meta_pshw,meta_sizes,meta_brights,meta_colors,meta_thicks,scores,helpfuls,uploadAts,collectAts
0,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,입어본 레이어드 티중에는 제일 두꺼운거같아요 단품도 예쁜데 지감은 추워서 레이어드용...,"남성, 171cm, 64kg",사이즈 보통이에요,밝기 밝아요,색감 선명해요,두께감 두꺼워요,width: 100%,0,18시간 전,2021-11-10 14:01:41.353825
1,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,동생 사줬는데 오버핏으로 딱 예뻐요 반팔 팔쪽에 포인트 원하시면 이걸로 가세요,"남성, 173cm, 67kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,0,1일 전,2021-11-10 14:01:41.357823
2,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,이쁩니다. 가격대비 무지 기본 이너티로써 최고의 선택이엇습니다. 다음에도 이용할게용~,"남성, 173cm, 60kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,0,2일 전,2021-11-10 14:01:41.359823
3,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,옷 질감도 부드럽고 일부러 크게입으려고 크게샀는데 사이즈도 낙낙하고 좋아요 이너로입기좋음,"남성, 185cm, 87kg",사이즈 커요,밝기 보통이에요,색감 보통이에요,두께감 두꺼워요,width: 100%,0,2일 전,2021-11-10 14:01:41.361821
4,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,옷 질감도 부드럽고 일부러 크게입으려고 크게샀는데 사이즈도 낙낙하고 좋아요 재구매임,"남성, 185cm, 87kg",사이즈 커요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,0,2일 전,2021-11-10 14:01:41.363820
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,TERRY HALF PANTS (MELANGE GREY) [LRPMCPH703MGYM],숏 팬츠,estimate_list,옷이좀크고두껍긴하지만여름에충분히입을수있어요.저는대만족,"남성, 177cm, 87kg",사이즈 커요,밝기 보통이에요,색감 선명해요,두께감 두꺼워요,width: 100%,0,2021.07.26,2021-11-10 20:10:00.705050
29996,TERRY HALF PANTS (MELANGE GREY) [LRPMCPH703MGYM],숏 팬츠,estimate_list,제가 좀 크게입는데 크게사도 편하고 입기 좋아요! 추천드림!,"남성, 178cm, 78kg",사이즈 보통이에요,밝기 밝아요,색감 선명해요,두께감 얇아요,width: 100%,0,2021.07.19,2021-11-10 20:10:00.706050
29997,TERRY HALF PANTS (MELANGE GREY) [LRPMCPH703MGYM],숏 팬츠,estimate_list,겨울용 인지 모르고 시켰네요.\n편하고 스타일 괜찮아서 그냥 겨울에 입을려구요,"여성, 162cm, 50kg",사이즈 커요,밝기 밝아요,색감 선명해요,두께감 두꺼워요,width: 100%,0,2021.07.01,2021-11-10 20:10:00.708034
29998,TERRY HALF PANTS (MELANGE GREY) [LRPMCPH703MGYM],숏 팬츠,estimate_list,사이즈 핏이 마음에 들고 색깔도 진하지도 연하지고 않고 이쁩니다.,"남성, 166cm, 60kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,0,2021.06.24,2021-11-10 20:10:00.710032


In [121]:
df['categories'].value_counts()

셔츠/블라우스       5000
숏 팬츠          5000
데님 팬츠         5000
반팔 티셔츠        5000
트레이닝/조거 팬츠    5000
슈트 팬츠/슬랙스     5000
Name: categories, dtype: int64

In [106]:
df['retypes'].value_counts()

style_estimate_list    9844
photo_estimate_list    5035
estimate_list          4321
Name: retypes, dtype: int64

In [ ]:
a = []
for p, m in zip(df['product_names'], df['meta_pshw']):
    if p == '주니어 스우시 반바지 BV6865-010':
        if m[0].isalpha():
            a.append('주니어 스우시 반바지 BV6865-010_1512387')
        else:
            a.append('주니어 스우시 반바지 BV6865-010_1293460')
    else:
        a.append(p)
df['product_names'] = a

In [107]:
a = df['product_names'].value_counts()
a

우먼즈 라운지 스웨트 팬츠 [블랙]                            100
BASIC HALF PANTS (RED) [LRPMCPH701MRDA]        100
우먼즈 슬림 스트레이트 히든 밴딩 슬랙스 [라이트 브라운]               100
우먼즈 웨이스트밴들리스 와이드 슬랙스 [블랙]                      100
사인로고 반팔 티셔츠 블랙                                 100
                                              ... 
Vegan Leather Two tuck Short pants               1
#signature [SHORTS.FIT] Carre jeans.622.pdf      1
우먼즈 오버사이즈 실키 셔츠 [블랙]                             1
우먼즈 오버사이즈 실키 셔츠 [화이트]                            1
STRAIGHT WIDE DENIM PANTS LIGHT BLUE             1
Name: product_names, Length: 289, dtype: int64

# 확인

In [122]:
df = pd.read_csv('crawling_20211110_positive.csv')

FileNotFoundError: [Errno 2] File crawling_20211110_positive.csv does not exist: 'crawling_20211110_positive.csv'

In [64]:
df

,Unnamed: 0,product_names,categories,retypes,rvs,meta_pshw,meta_sizes,meta_brights,meta_colors,meta_thicks,scores,helpfuls,uploadAts,collectAts
0,0,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,길어서 후드 밑에 받쳐입어야할 것 같아요\n약간 가성비..?! 두꺼워서 가을에입기 ...,"여성, 161cm, 57kg",사이즈 커요,밝기 보통이에요,색감 보통이에요,두께감 두꺼워요,width: 60%,0,3시간 전,2021-11-04 23:35:35.399930
1,1,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,오버핏으로 레이어드 해서 입는걸 좋아하는데\n다른 코튼 티셔츠들보다 목부분이 튼튼하...,"여성, 158cm, 50kg",사이즈 커요,밝기 보통이에요,색감 선명해요,두께감 두꺼워요,width: 100%,0,1일 전,2021-11-04 23:35:35.401920
2,2,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,길이감이 좋아서 어디 받쳐입기 굉장히 좋아요 그리고 적당히 두꺼워서 맨살이 안 비쳐요!,"남성, 178cm, 68kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2일 전,2021-11-04 23:35:35.402919
3,3,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,길이감 좋고 두께도 적당해서 레이어드 하기 좋아요!,"남성, 178cm, 68kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2일 전,2021-11-04 23:35:35.404927
4,4,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,스타일 후기,사이즈잘맞아요 핏도좋습니다 감사합니다 잘입을게요,"남성, 175cm, 65kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,0,3일 전,2021-11-04 23:35:35.405932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29995,UNBALANCE SHORT PANTS [ICE],숏 팬츠,일반 후기,예전에 M사이즈 잘못구매해서 사놓구 교환하는거 잊고있다가 이만한 색상 핏 길이 등등...,NaN,사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,1,2020.08.06,2021-11-05 05:24:11.772895
29996,29996,UNBALANCE SHORT PANTS [ICE],숏 팬츠,일반 후기,두께 있으면서 얇아서 시원해요\n뭔말인가 싶지만ㅋㅋㅋ 얇아서 주름이 좀 잘 생기고 ...,NaN,NaN,NaN,NaN,NaN,width: 100%,1,2019.08.24,2021-11-05 05:24:11.772895
29997,29997,UNBALANCE SHORT PANTS [ICE],숏 팬츠,일반 후기,가격도 싸고 쿠폰기간얼마안남아서 쿠폰쓸겨 겸사겸사사봤아여..\n색이 겁나연청이구여 ...,NaN,NaN,NaN,NaN,NaN,width: 100%,4,2018.07.01,2021-11-05 05:24:11.773894
29998,29998,UNBALANCE SHORT PANTS [ICE],숏 팬츠,일반 후기,예전에 M사이즈 잘못구매해서 사놓구 교환하는거 잊고있다가 이만한 색상 핏 길이 등등...,NaN,사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 100%,1,2020.08.06,2021-11-05 05:24:13.377814


In [ ]:
from collections import Counter

In [108]:
cntr = Counter(df.rvs)

In [109]:
sorted(cntr.items(), key=lambda x:x[1], reverse=True)

[('짧은 길이를 원해서 S을 시켰는데 괜찮은것 같아요! 너비고 괜찮고 두깨도 딱 적당합니다. 다만 색상이 조금만 더 짙었으면 더 좋았을듯 합니다잉. 로고는 밑에 작게 예쁘게 나와요!',
  18),
 ('바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임', 17),
 ('평소 27 입는데 허벅지나 엉덩이 기장은 좋은데 허리가 살짝만 작았으면 딱 좋았을 것 같아요 밴딩은 크게 효과는 모르겠지만 재질이 탄탄해서 좋아요',
  16),
 ('가성비 좋아요 잘입어요\n두께감도있고 편해요\n감사합니다', 16),
 ('전에 구매했는데 편해서 하나 더 샀어요\n생각보다 허벅지 실측이 커서 다음 구매에는 하나 작은 사이즈 사도 될 거 같음', 16),
 ('생각보다 질 괜찮네요. 자주 입고 다닐 것 같습니다.', 16),
 ('핏 좋아요 생각보다 두꺼워서 여름에는 좀 더울거같아요', 16),
 ('한여름 말고는 다 입을 수 있을 수 있겠다 싶을 정도로 두께가 있는 바지네요.\n편하게 늘어나지만 회복력도 좋다는~', 12),
 ('핏이 딱 맞아서 입었을 때 편해요. 허리 밴딩과 신축성 모두 좋고, 제가 원하던 그레이 색감이라 더 만족스럽습니다.', 12),
 ('여기서 연청30도 구매하였는데 사이즈딱이였어요 근데 왜... 이녀석은 허리가 엄청나게 컸을까요...(수선집달려감.)', 12),
 ('사이즈가 좀 아쉽습니다\n그래도 크림색 바지 하나 장만하니 좋네요\n주머니가 포인트입니다', 12),
 ('짧은 길이를 원해서 S을 시켰는데 괜찮은것 같아요! 너비고 괜찮고 두깨도 딱 적당합니다. 다만 색상이 조금만 더 어두웠으면 더 좋았을듯 합니다잉.',
  12),
 ('얇아서 한여름에도 입기 괜찮을거 같고 길이도 적당해서 마음에 들어용', 12),
 ('싼데 나쁘지 않아요 이런게 킹성비라고 할수 있죠\n가격대비 좋다는거지 엄청 고품질의 옷은 아닙니다', 12),
 ('반바지 필요해서 샀는데 편하고 좋아요 딱 맞고 입기 좋은용도', 12),
 ('싸고 질도 

In [18]:
df[df.product_names == 'UNBALANCE SHORT PANTS [ICE]']

,Unnamed: 0,product_names,categories,retypes,rvs,meta_pshw,meta_sizes,meta_brights,meta_colors,meta_thicks,scores,helpfuls,uploadAts,collectAts


In [110]:
df[df.rvs == '바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임']

,product_names,categories,retypes,rvs,meta_pshw,meta_sizes,meta_brights,meta_colors,meta_thicks,scores,helpfuls,uploadAts,collectAts
16499,BASIC HALF PANTS (BLACK) [LRPMCPH701MBKA],숏 팬츠,style_estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 03:52:27.476079
16533,BASIC HALF PANTS (BLACK) [LRPMCPH701MBKA],숏 팬츠,photo_estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 03:53:00.180502
16594,BASIC HALF PANTS (BLACK) [LRPMCPH701MBKA],숏 팬츠,estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 03:54:08.508210
16985,BASIC HALF PANTS (BEIGE) [LRPMCPH701MBEA],숏 팬츠,style_estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 04:04:57.179843
17019,BASIC HALF PANTS (BEIGE) [LRPMCPH701MBEA],숏 팬츠,photo_estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 04:05:29.712494
17080,BASIC HALF PANTS (BEIGE) [LRPMCPH701MBEA],숏 팬츠,estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 04:06:35.916988
17446,BASIC HALF PANTS (KHAKI) [LRPMCPH701MKHA],숏 팬츠,style_estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 04:13:21.723770
17480,BASIC HALF PANTS (KHAKI) [LRPMCPH701MKHA],숏 팬츠,photo_estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 04:13:53.017675
17541,BASIC HALF PANTS (KHAKI) [LRPMCPH701MKHA],숏 팬츠,estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 04:14:59.163045
18227,BASIC HALF PANTS (LIGHT KHAKI) [LRPMCPH701MKHL],숏 팬츠,style_estimate_list,바지 생각보다 진짜 갠차늠 나는 아주 만족스러움 그래서 잘 입을 예정임,"남성, 179cm, 82kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,2021.08.15,2021-11-10 04:29:47.124198


In [111]:
df

,product_names,categories,retypes,rvs,meta_pshw,meta_sizes,meta_brights,meta_colors,meta_thicks,scores,helpfuls,uploadAts,collectAts
0,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,길어서 후드 밑에 받쳐입어야할 것 같아요\n약간 가성비..?! 두꺼워서 가을에입기 ...,"여성, 161cm, 57kg",사이즈 커요,밝기 보통이에요,색감 보통이에요,두께감 두꺼워요,width: 60%,0,4일 전,2021-11-09 19:47:15.435477
1,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,길이감이 좋아서 어디 받쳐입기 굉장히 좋아요 그리고 적당히 두꺼워서 맨살이 안 비쳐요!,"남성, 178cm, 68kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,7일 전,2021-11-09 19:47:15.436477
2,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,길이감 좋고 두께도 적당해서 레이어드 하기 좋아요!,"남성, 178cm, 68kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,7일 전,2021-11-09 19:47:15.439491
3,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,좋습니다!\n근데 제가 생각이 짧아 미처 수축을 생각을 못했네요..\nM시켰는데 받...,"남성, 173cm, 70kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 보통이에요,width: 80%,0,11일 전,2021-11-09 19:47:15.441473
4,[세트상품] 더블 코튼 롱 티셔츠,반팔 티셔츠,style_estimate_list,여름이면 기본적으로 한세트씩 주문하는 기본템입니다. 그만큼 안이나 밖 둘다 입기 너...,"남성, 186cm, 89kg",사이즈 커요,밝기 보통이에요,색감 선명해요,두께감 보통이에요,width: 80%,0,12일 전,2021-11-09 19:47:18.665858
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,BASIC HALF PANTS (LIGHT BEIGE) [LRPMCPH701MBEL],숏 팬츠,estimate_list,몇번 안입었는데 허벅지 사이에 보풀이 잘 일어나네요,"남성, 170cm, 66kg",사이즈 보통이에요,밝기 밝아요,색감 보통이에요,두께감 보통이에요,width: 60%,0,2021.09.29,2021-11-10 04:50:57.500976
19196,BASIC HALF PANTS (LIGHT BEIGE) [LRPMCPH701MBEL],숏 팬츠,estimate_list,기본아이템으로 가지고 있을만해요 충분히 만족합니다,,사이즈 보통이에요,밝기 보통이에요,색감 선명해요,두께감 보통이에요,width: 80%,0,2021.09.29,2021-11-10 04:50:57.501976
19197,BASIC HALF PANTS (LIGHT BEIGE) [LRPMCPH701MBEL],숏 팬츠,estimate_list,색상도 이뿌고 짧은 반바지 원했는데 살짝 짧아서 좋아요\n전체적으로 마음에 들어요,"남성, 171cm, 71kg",사이즈 보통이에요,밝기 밝아요,색감 선명해요,두께감 얇아요,width: 80%,0,2021.09.23,2021-11-10 04:50:57.503974
19198,BASIC HALF PANTS (LIGHT BEIGE) [LRPMCPH701MBEL],숏 팬츠,estimate_list,괜찮은데 생각보다 길이랑 두께가 얇네요 아주 그리고 그러가보니 구김이 많이거요,"여성, 165cm, 58kg",사이즈 보통이에요,밝기 보통이에요,색감 보통이에요,두께감 얇아요,width: 40%,0,2021.09.01,2021-11-10 04:51:01.255626


In [120]:
sum(df.duplicated())

0